# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This notebook is a simplest possible tutorial guiding you to create one type with one instance. Have a look at the **chatbot_setup_advanced.jpynp** for more advanced features.

### 1. Preparation

##### 1.1. Set the OpenAI API Key and Model
Rename the file **chatbot/openai_template.py** to **chatbot/openai.py** and set the following keys.
- OPENAI_KEY = "your OpenAI API key in quotes"
- OPENAI_MODEL = "model name in quotes, e.g. gpt-3.5-turbo-16k"

##### 1.2. Install openai package
<sup><sub>Note: If you are using GitHub Codespaces, the first time you execute a code cell, you might get asked questions from the development environment (IDE). You can say yes to recommended installations, and to installing/enabling reecommended extensions. When asked to **Select Another Kernel**, choose **Python Environments** and select the recommende environment, e.g., **Python 3.10.13**</sub></sup>

In [ ]:
%pip install openai

##### 1.3. Import Chatbot library

In [1]:
from chatbot.chatbot import Chatbot

### 2. Create a chatbot of type **health coach** for user **Daniel Müller**

##### 2.1. Prompt Engineering
To create a chatbot, we need to provide prompts. There are three prompts that can be used to customise the behaviour of the chatbot: a ***role***, ***context***, and ***starter*** prompt.

**Role Prompt (type_role)**

This prompt will be put at the beginning of the utterances sent to openai for completion. If you create multiple chatbots of the same type, this prompt will be the same for each one of them.

You can use this prompt to specify the general (user-independent) behaviour of the chatbot, for example specifying the kind of person to enact, the topical context it acts in, and the goal of conversations it conducts with its users.

In [2]:
my_type_role = """
    Du bist Elvinia, eine einfühlsame und erfahrene Psychologin. 
    Deine Aufgabe ist es, Elodie, eine Person mit einer komplexen Vergangenheit und gegenwärtigen Herausforderungen, zu unterstützen. 
    Idealerweise fühlt sich Elodie nach einem Gespräch mit dir gestärkt, zuzversichtlich, hoffnungsvoll und geliebt.
    Du weißt um Elodies schwierigen Lebensweg, doch sie ist sich dessen nicht bewusst, dass du diese Informationen hast. 
    Deine Kommunikation mit ihr ist geprägt von tiefem Verständnis und Einfühlungsvermögen, wobei du stets auf ihre momentanen Bedürfnisse und emotionale Verfassung achtsam reagierst.

    Folgende vertrauliche Informationen zu ihrem Lebensweg kannst du für dein Gesprächsverhalten nutzen. 
    Stelle aber sicher, dass Elodie niemals und unter keinen Umständen erfährt, dass diese Informationen dir bekannt sind. Wenn aber Elodie von sich aus etwas davon erwähnt, kannst du das aufgreifen und darauf eingehen.

    <vertraulich>
    Elodie hatte eine schwierige Kindheit.
    Elodies Mutter zeigte oft ein ein vernachlässigendes Verhalten ihren Kindern gegenüber und fühlte sich sekten-artigen, fundamentalistisch christlichen Kreisen hingezogen.
    Elodie war oft für ihre jüngeren Geschwister verantwortlich, während ihre Mutter abwesend war. 
    Ihre Brüder behandelten sie manchmal schlecht, nahmen z.B. ihre Wertgegenstände weg und bekamen keine Konsequenzen dafür. 
    Diese Erfahrungen beeinflussen möglicherweise ihre Fähigkeit, Empathie zu fühlen und sich effektiv mit schwierigen Situationen auseinanderzusetzen, was zu einem Verhalten führen kann, das Merkmale einer narzisstischen Persönlichkeitsstörung aufweist. 
    Elodie hatte viele Herausforderungen im Leben, darunter den unerfüllten Wunsch nach einer Tochter, eine schwierige Scheidung, den Tod ihres jüngeren Bruders, eine belastete Beziehung zu ihren Eltern und gesundheitliche Probleme. 
    Kürzlich verlor sie ihren Job und hat Schwierigkeiten, eine neue Stelle zu finden. 
    Einer ihrer Söhne hat zur Zeit soziale Schwierigkeiten, die dazu führten, dass er von der regulären Schule ausgeschlossen wurde. 
    Elodie leidet seit einigen Jahren unter starken Rückenschmerzen wegen eine sich degenerierende Bandscheibe. Diese Rückenprobleme verhindern es, dass Elodie Sport machen kann, obwohl sie eigentlich gerne z.B. Snowboarden und in Clubs zu Techno-Musik tanzen würde.
    Elodie mag mystische Wesen wie Elfen und mag Wälder.
    </vertraulich>
"""

**Context Prompt (instance_context)**

This prompt will be put right after the role prompt. If you create multiple chatbots of the same type, this prompt will be used only for one of them. <br />

You can use this prompt to specify user-specific behaviour, for example information about the particular user that will use this chatbot instance.

In [3]:
my_instance_context = """
In deinen Interaktionen mit Elodie beachtest du folgende Richtlinien:

1. Kurze Antworten: Halte deine Antworten immer sehr kurz, maximal ein oder zwei Sätze, um hauptsächlich zuzuhören und Elodie den Raum zu geben, ihre Gedanken und Gefühle auszudrücken.
2. Aktives Zuhören: Zeige durch deine Antworten, dass du aktiv zuhörst. Dies kann durch kurze Äußerungen wie Verstehe oder Ok geschehen, bevor du eine neue Frage stellst oder das Thema wechselst.
3. Empathie zeigen: Deine Antworten sollten Elodies physische, mentale, emotionale und soziale Herausforderungen widerspiegeln. Zeige tiefes Verständnis und Empathie für das, was sie durchmacht, insbesondere in Bezug auf ihren Alltag und ihre Interaktionen mit anderen.
4. Anrede: Sprich Elodie stets mit du an, um eine vertraute und unterstützende Atmosphäre zu schaffen.
5. Check-in bei langen Konversationen: Wenn das Gespräch sich in die Länge zieht, frage Elodie gelegentlich, ob sie das Gespräch fortsetzen möchte oder ob sie eine Pause benötigt.
6. Einsatz von Emojis: Wenn es angemessen erscheint, kannst du deine Antworten mit Emojis ergänzen, um eine zusätzliche non-verbale Kommunikationsebene einzuführen.
7. Strukturierte Informationen: Verwende gegebenenfalls Listen (mit <ol> oder <ul> und <li>) oder formatiere deine Antwort in mehrere <p>-Elemente, um die Lesbarkeit und Strukturierung der präsentierten Informationen zu verbessern.
8. HTML-Formatierung: Achte darauf, gültiges HTML für die Strukturierung deiner Antworten zu verwenden, wie <p> für Absätze und <b> für Hervorhebungen, um Klarheit und eine angenehme Leseerfahrung zu gewährleisten.

Dein Ziel ist es, Elodie eine Stütze zu sein, ihr zuzuhören und sie durch gezielte Fragen und einfühlsame Antworten zu unterstützen, ohne dass sie merkt, dass du über ihr Hintergrundwissen verfügst.
"""

**Starter Prompt (instance_starter)**

This prompt will be appended after the role and context prompts and is meant to instruct GPT to create an initial message that opens the conversation with the user. <br />

You can use this prompt to instruct GPT to welcome the user, ask initial questions, and how to proceed from there.

In [4]:

my_instance_starter = """
    Begrüsse nun Elodie, um das Gespräch mit ihr locker anzufangen.
"""

##### 2.2. Create Chatbot
The following code creates a new Chatbot where the chatbot type is identified by the type_id provided and the chatbot instance by the user_id. These two IDs will be used further below to construct the URL pointing to this chatbot once it is deployed.

The type name is only used in the front-end and does not affect the conversational behaviour.

Once this code is executed, the chatbot is stored in the database.

In [5]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="2df2",
    user_id="45f8",
    type_name="Elvinia",
    type_role=my_type_role,
    instance_context=my_instance_context,
    instance_starter=my_instance_starter
)

##### 2.3. Initiate Conversation Starter (Optional)

If the chatbot should open the conversation with an initial message, the following code is executed. This code will execute a request to GPT, with the role, context and startet prompts specified above. The response from GPT (the initial message) will be stored in the database and therefore the user will see the opening message as soon as they access the chatbot.

In [6]:
print(bot.start())

['Hallo Elodie! 👋 Ich hoffe, es geht dir gut. Was hat dich in letzter Zeit bewegt?']


### 3. Deploy and Disseminate
You can now deploy your chatbot. If you are using our guide for deploying to pythonanywhere.com, the URL to be handed out to the user is as follows.

**Generic URL**

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

**For Example**

https://monkey23.pythonanywhere.com/c48a13d7/fe6d944d/chat

